In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgbm
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, make_scorer
from sklearn.ensemble import BaggingClassifier
import re
import seaborn as sns
from IPython.display import Image
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_validate

In [2]:
train = pd.read_csv('./data/train.csv', index_col=0)
test = pd.read_csv('./data/test_x.csv', index_col=0)
submission = pd.read_csv('./data/sample_submission.csv', index_col=0)  

### 설문조사 소요시간을 사람별로 정규화

In [3]:
survey_time = []
for col in list(train.columns):
    if re.match('Q[a-z]E', col):
        survey_time.append(col)
train[survey_time] = train[survey_time].apply(lambda x: x / x.max(), axis=1)   

survey_time = []
for col in list(test.columns):
    if re.match('Q[a-z]E', col):
        survey_time.append(col)
test[survey_time] = test[survey_time].apply(lambda x: x / x.max(), axis=1)    

In [4]:
train.head(5)

,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,...,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13
index,,,,,,,,,,,,,,,,,,,,,
0,3.0,0.230184,4.0,0.868738,5.0,0.632213,1.0,0.649334,2.0,1.000000,...,0,1,0,1,1,0,1,0,1,1
1,5.0,0.149457,5.0,0.303303,3.0,0.782398,5.0,0.685840,1.0,0.997921,...,1,1,0,1,1,0,1,0,1,1
2,4.0,0.481031,1.0,0.438648,1.0,0.302608,4.0,1.000000,5.0,0.395080,...,1,1,0,1,1,1,1,0,1,1
3,3.0,0.021116,3.0,0.096824,4.0,0.041562,3.0,0.135956,1.0,0.014957,...,0,0,0,0,1,0,1,0,1,1
4,1.0,0.490736,1.0,0.374272,5.0,0.294336,2.0,0.562202,1.0,0.536792,...,1,1,1,1,1,0,1,1,1,1


In [5]:
test.head(5)

,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,...,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13
index,,,,,,,,,,,,,,,,,,,,,
0,3.0,0.082780,2.0,0.330784,3.0,0.519739,1.0,0.546283,2.0,0.286807,...,0,1,0,0,1,0,1,0,1,1
1,3.0,0.113591,2.0,0.431381,3.0,0.342983,3.0,0.181436,4.0,0.254144,...,0,0,0,0,0,0,0,0,0,0
2,3.0,0.150739,2.0,0.755803,4.0,0.144709,2.0,0.185107,2.0,0.399759,...,0,1,0,1,1,0,1,0,1,1
3,1.0,0.231809,1.0,0.363825,5.0,0.497228,2.0,0.780319,5.0,0.877685,...,1,1,1,1,1,1,1,1,1,1
4,2.0,0.256555,1.0,0.639075,5.0,0.434447,2.0,0.856555,2.0,0.475578,...,1,1,0,1,1,0,1,1,1,1


### 마키아벨리즘

In [6]:
Answers = ['QaA', 'QbA', 'QcA', 'QdA', 'QeA',
             'QfA', 'QgA', 'QhA', 'QiA', 'QjA', 
             'QkA', 'QlA', 'QmA', 'QnA', 'QoA', 
             'QpA', 'QqA', 'QrA', 'QsA', 'QtA']

In [7]:
flipping_columns = ["QeA", "QfA", "QkA", "QqA", "QrA", "QaA", "QdA", "QgA", "QiA", "QnA"]
for flip in flipping_columns: 
    train[flip] = 6 - train[flip]
    test[flip] = 6 - test[flip]
train['Mach_score'] = train[Answers].mean(axis = 1)
test['Mach_score'] = test[Answers].mean(axis = 1)

In [8]:
train.head()

,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,...,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13,Mach_score
index,,,,,,,,,,,,,,,,,,,,,
0,3.0,0.230184,4.0,0.868738,5.0,0.632213,5.0,0.649334,4.0,1.000000,...,1,0,1,1,0,1,0,1,1,2.95
1,1.0,0.149457,5.0,0.303303,3.0,0.782398,1.0,0.685840,5.0,0.997921,...,1,0,1,1,0,1,0,1,1,2.60
2,2.0,0.481031,1.0,0.438648,1.0,0.302608,2.0,1.000000,1.0,0.395080,...,1,0,1,1,1,1,0,1,1,1.90
3,3.0,0.021116,3.0,0.096824,4.0,0.041562,3.0,0.135956,5.0,0.014957,...,0,0,0,1,0,1,0,1,1,3.35
4,5.0,0.490736,1.0,0.374272,5.0,0.294336,4.0,0.562202,5.0,0.536792,...,1,1,1,1,0,1,1,1,1,3.00


In [9]:
test.head()

,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,...,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13,Mach_score
index,,,,,,,,,,,,,,,,,,,,,
0,3.0,0.082780,2.0,0.330784,3.0,0.519739,5.0,0.546283,4.0,0.286807,...,1,0,0,1,0,1,0,1,1,2.45
1,3.0,0.113591,2.0,0.431381,3.0,0.342983,3.0,0.181436,2.0,0.254144,...,0,0,0,0,0,0,0,0,0,2.80
2,3.0,0.150739,2.0,0.755803,4.0,0.144709,4.0,0.185107,4.0,0.399759,...,1,0,1,1,0,1,0,1,1,3.55
3,5.0,0.231809,1.0,0.363825,5.0,0.497228,4.0,0.780319,1.0,0.877685,...,1,1,1,1,1,1,1,1,1,3.35
4,4.0,0.256555,1.0,0.639075,5.0,0.434447,4.0,0.856555,4.0,0.475578,...,1,0,1,1,0,1,1,1,1,2.90


In [10]:
train_x = train.drop('voted', axis = 1)
train_y = train['voted']

In [11]:
X_train, X_vali, Y_train, Y_vali = train_test_split(train_x, train_y, test_size=0.25)

In [12]:
train_x_dummies = pd.get_dummies(X_train)
vali_x_dummies = pd.get_dummies(X_vali)

all_train_x_dummies = pd.get_dummies(train_x)
test_x_dummies = pd.get_dummies(test)

In [13]:
train_x_dummies.shape, vali_x_dummies.shape, all_train_x_dummies.shape, test_x_dummies.shape

((34149, 101), (11383, 101), (45532, 101), (11383, 101))

- train_val

In [14]:
lgbm_model = lgbm.LGBMClassifier(n_estimators=500)
lgbm_model.fit(train_x_dummies, Y_train)

LGBMClassifier(n_estimators=500)

In [15]:
roc_auc_score(Y_vali, list(pd.DataFrame(lgbm_model.predict_proba(vali_x_dummies)).apply(lambda x: x[1], axis=1)))

0.7567656823614162

- train_test

In [ ]:
lgbm_model2 = lgbm.LGBMClassifier(n_estimators=500)
bagging_lgbm_model = BaggingClassifier(base_estimator=lgbm_model2, n_estimators=30).fit(all_train_x_dummies, train_y)

In [ ]:
submission['voted']= list(pd.DataFrame(bagging_lgbm_model.predict_proba(test_x_dummies)).apply(lambda x: x[1], axis=1))

In [ ]:
submission.to_csv('./data/bagging_lgbm.csv')

### lgbm 파라미터 튜닝

In [30]:
def lgbm_cv(learning_rate, num_leaves, max_depth, min_child_weight, colsample_bytree, feature_fraction, bagging_fraction, lambda_l1, lambda_l2):
    model = lgbm.LGBMClassifier(learning_rate=learning_rate,
                                n_estimators = 300,
                                #boosting = 'dart',
                                num_leaves = int(round(num_leaves)),
                                max_depth = int(round(max_depth)),
                                min_child_weight = int(round(min_child_weight)),
                                colsample_bytree = colsample_bytree,
                                feature_fraction = max(min(feature_fraction, 1), 0),
                                bagging_fraction = max(min(bagging_fraction, 1), 0),
                                lambda_l1 = max(lambda_l1, 0),
                                lambda_l2 = max(lambda_l2, 0)
                               )
    scoring = {'roc_auc_score': make_scorer(roc_auc_score, needs_proba=True)}
    result = cross_validate(model, all_train_x_dummies, train_y, cv=5, scoring=scoring)
    auc_score = result["test_roc_auc_score"].mean()
    return auc_score

In [31]:
pbounds = {'learning_rate' : (0.0001, 0.05),
           'num_leaves': (300, 600),
           'max_depth': (2, 25),
           'min_child_weight': (30, 100),
           'colsample_bytree': (0, 0.99),
           'feature_fraction': (0.0001, 0.99),
           'bagging_fraction': (0.0001, 0.99),
           'lambda_l1' : (0, 0.99),
           'lambda_l2' : (0, 0.99),
          }

In [32]:
lgbmBO = BayesianOptimization(f = lgbm_cv, pbounds = pbounds, verbose = 2, random_state = 0 )

In [33]:
lgbmBO.maximize(init_points=5, n_iter = 20, acq='ei', xi=0.01)

|   iter    |  target   | baggin... | colsam... | featur... | lambda_l1 | lambda_l2 | learni... | max_depth | min_ch... | num_le... |
-------------------------------------------------------------------------------------------------------------------------------------
[LightGBM] [Warning] feature_fraction is set=0.5967754659733202, colsample_bytree=0.7080374727086953 will be ignored. Current value: feature_fraction=0.5967754659733202
[LightGBM] [Warning] lambda_l1 is set=0.5394343511669278, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5394343511669278
[LightGBM] [Warning] bagging_fraction is set=0.5433704875376587, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5433704875376587
[LightGBM] [Warning] lambda_l2 is set=0.41941825134551564, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.41941825134551564
[LightGBM] [Warning] feature_fraction is set=0.5967754659733202, colsample_bytree=0.7080374727086953 will be ignored. Current value: feature_fracti

[LightGBM] [Warning] feature_fraction is set=0.968834296976213, colsample_bytree=0.8613120267643509 will be ignored. Current value: feature_fraction=0.968834296976213
[LightGBM] [Warning] lambda_l1 is set=0.7911669785745563, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.7911669785745563
[LightGBM] [Warning] bagging_fraction is set=0.770397367765257, subsample=1.0 will be ignored. Current value: bagging_fraction=0.770397367765257
[LightGBM] [Warning] lambda_l2 is set=0.45686456863040253, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.45686456863040253
[LightGBM] [Warning] feature_fraction is set=0.968834296976213, colsample_bytree=0.8613120267643509 will be ignored. Current value: feature_fraction=0.968834296976213
[LightGBM] [Warning] lambda_l1 is set=0.7911669785745563, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.7911669785745563
[LightGBM] [Warning] bagging_fraction is set=0.770397367765257, subsample=1.0 will be ignored. Current value: bagging_f

[LightGBM] [Warning] feature_fraction is set=0.026208788981509707, colsample_bytree=0.5591533333920333 will be ignored. Current value: feature_fraction=0.026208788981509707
[LightGBM] [Warning] lambda_l1 is set=0.17381298410693863, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.17381298410693863
[LightGBM] [Warning] bagging_fraction is set=0.9620869765251262, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9620869765251262
[LightGBM] [Warning] lambda_l2 is set=0.8315034502552536, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.8315034502552536
[LightGBM] [Warning] feature_fraction is set=0.026208788981509707, colsample_bytree=0.5591533333920333 will be ignored. Current value: feature_fraction=0.026208788981509707
[LightGBM] [Warning] lambda_l1 is set=0.17381298410693863, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.17381298410693863
[LightGBM] [Warning] bagging_fraction is set=0.9620869765251262, subsample=1.0 will be ignored. Current

|  8        |  0.769    |  0.8959   |  0.1059   |  0.7973   |  0.5175   |  0.1686   |  0.02376  |  4.552    |  38.16    |  318.1    |
[LightGBM] [Warning] feature_fraction is set=0.8970263965758809, colsample_bytree=0.15461802223371843 will be ignored. Current value: feature_fraction=0.8970263965758809
[LightGBM] [Warning] lambda_l1 is set=0.1735538721469622, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1735538721469622
[LightGBM] [Warning] bagging_fraction is set=0.17726022119687482, subsample=1.0 will be ignored. Current value: bagging_fraction=0.17726022119687482
[LightGBM] [Warning] lambda_l2 is set=0.653280162273047, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.653280162273047
[LightGBM] [Warning] feature_fraction is set=0.8970263965758809, colsample_bytree=0.15461802223371843 will be ignored. Current value: feature_fraction=0.8970263965758809
[LightGBM] [Warning] lambda_l1 is set=0.1735538721469622, reg_alpha=0.0 will be ignored. Current value: lambda_l

[LightGBM] [Warning] feature_fraction is set=0.26698030969606845, colsample_bytree=0.2890613505296128 will be ignored. Current value: feature_fraction=0.26698030969606845
[LightGBM] [Warning] lambda_l1 is set=0.8589055845838335, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.8589055845838335
[LightGBM] [Warning] bagging_fraction is set=0.8057828508386722, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8057828508386722
[LightGBM] [Warning] lambda_l2 is set=0.5556391935581778, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5556391935581778
[LightGBM] [Warning] feature_fraction is set=0.26698030969606845, colsample_bytree=0.2890613505296128 will be ignored. Current value: feature_fraction=0.26698030969606845
[LightGBM] [Warning] lambda_l1 is set=0.8589055845838335, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.8589055845838335
[LightGBM] [Warning] bagging_fraction is set=0.8057828508386722, subsample=1.0 will be ignored. Current value: 

[LightGBM] [Warning] feature_fraction is set=0.7260153999535411, colsample_bytree=0.43994370627537954 will be ignored. Current value: feature_fraction=0.7260153999535411
[LightGBM] [Warning] lambda_l1 is set=0.065596345441734, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.065596345441734
[LightGBM] [Warning] bagging_fraction is set=0.9279740124811896, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9279740124811896
[LightGBM] [Warning] lambda_l2 is set=0.9837684135123563, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.9837684135123563
[LightGBM] [Warning] feature_fraction is set=0.7260153999535411, colsample_bytree=0.43994370627537954 will be ignored. Current value: feature_fraction=0.7260153999535411
[LightGBM] [Warning] lambda_l1 is set=0.065596345441734, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.065596345441734
[LightGBM] [Warning] bagging_fraction is set=0.9279740124811896, subsample=1.0 will be ignored. Current value: baggin

|  16       |  0.7688   |  0.7129   |  0.9263   |  0.8627   |  0.2539   |  0.7501   |  0.01292  |  13.36    |  91.28    |  544.6    |
[LightGBM] [Warning] feature_fraction is set=0.042687714774223895, colsample_bytree=0.5155879430764508 will be ignored. Current value: feature_fraction=0.042687714774223895
[LightGBM] [Warning] lambda_l1 is set=0.9729219107042695, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.9729219107042695
[LightGBM] [Warning] bagging_fraction is set=0.897929274006192, subsample=1.0 will be ignored. Current value: bagging_fraction=0.897929274006192
[LightGBM] [Warning] lambda_l2 is set=0.0534203079043078, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0534203079043078
[LightGBM] [Warning] feature_fraction is set=0.042687714774223895, colsample_bytree=0.5155879430764508 will be ignored. Current value: feature_fraction=0.042687714774223895
[LightGBM] [Warning] lambda_l1 is set=0.9729219107042695, reg_alpha=0.0 will be ignored. Current value: lamb

[LightGBM] [Warning] feature_fraction is set=0.32446975419842866, colsample_bytree=0.8269315384301569 will be ignored. Current value: feature_fraction=0.32446975419842866
[LightGBM] [Warning] lambda_l1 is set=0.7114727635499094, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.7114727635499094
[LightGBM] [Warning] bagging_fraction is set=0.01392510287568581, subsample=1.0 will be ignored. Current value: bagging_fraction=0.01392510287568581
[LightGBM] [Warning] lambda_l2 is set=0.16350938695169082, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.16350938695169082
[LightGBM] [Warning] feature_fraction is set=0.32446975419842866, colsample_bytree=0.8269315384301569 will be ignored. Current value: feature_fraction=0.32446975419842866
[LightGBM] [Warning] lambda_l1 is set=0.7114727635499094, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.7114727635499094
[LightGBM] [Warning] bagging_fraction is set=0.01392510287568581, subsample=1.0 will be ignored. Current va

[LightGBM] [Warning] feature_fraction is set=0.6863019215645269, colsample_bytree=0.812282013961385 will be ignored. Current value: feature_fraction=0.6863019215645269
[LightGBM] [Warning] lambda_l1 is set=0.3137806579851995, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.3137806579851995
[LightGBM] [Warning] bagging_fraction is set=0.296609291028754, subsample=1.0 will be ignored. Current value: bagging_fraction=0.296609291028754
[LightGBM] [Warning] lambda_l2 is set=0.13692638726425754, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.13692638726425754
[LightGBM] [Warning] feature_fraction is set=0.6863019215645269, colsample_bytree=0.812282013961385 will be ignored. Current value: feature_fraction=0.6863019215645269
[LightGBM] [Warning] lambda_l1 is set=0.3137806579851995, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.3137806579851995
[LightGBM] [Warning] bagging_fraction is set=0.296609291028754, subsample=1.0 will be ignored. Current value: bagging

|  24       |  0.7674   |  0.1586   |  0.04466  |  0.3328   |  0.545    |  0.392    |  0.0325   |  24.1     |  97.87    |  536.9    |
[LightGBM] [Warning] feature_fraction is set=0.2883372596410287, colsample_bytree=0.08468470457209575 will be ignored. Current value: feature_fraction=0.2883372596410287
[LightGBM] [Warning] lambda_l1 is set=0.6727416236760422, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.6727416236760422
[LightGBM] [Warning] bagging_fraction is set=0.5143528555810121, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5143528555810121
[LightGBM] [Warning] lambda_l2 is set=0.49735621820755394, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.49735621820755394
[LightGBM] [Warning] feature_fraction is set=0.2883372596410287, colsample_bytree=0.08468470457209575 will be ignored. Current value: feature_fraction=0.2883372596410287
[LightGBM] [Warning] lambda_l1 is set=0.6727416236760422, reg_alpha=0.0 will be ignored. Current value: lambda

In [34]:
lgbmBO.max

{'target': 0.7689795171044522,
 'params': {'bagging_fraction': 0.8958797357815458,
  'colsample_bytree': 0.10593113961701743,
  'feature_fraction': 0.7973403266487863,
  'lambda_l1': 0.5174759248593073,
  'lambda_l2': 0.16855495537647786,
  'learning_rate': 0.023759205615142862,
  'max_depth': 4.5520687487038725,
  'min_child_weight': 38.15818358905622,
  'num_leaves': 318.0864399129222}}

In [35]:
tuned_lgbm_model = lgbm.LGBMClassifier(learning_rate=lgbmBO.max['params']['learning_rate'],
                               num_leaves = int(round(lgbmBO.max['params']['num_leaves'])),
                               max_depth = int(round(lgbmBO.max['params']['max_depth'])),
                               min_child_weight = int(round(lgbmBO.max['params']['min_child_weight'])),
                               colsample_bytree=lgbmBO.max['params']['colsample_bytree'],
                               feature_fraction = max(min(lgbmBO.max['params']['feature_fraction'], 1), 0),
                               bagging_fraction = max(min(lgbmBO.max['params']['bagging_fraction'], 1), 0),
                               lambda_l1 = lgbmBO.max['params']['lambda_l1'],
                               lambda_l2 = lgbmBO.max['params']['lambda_l2']
                               )

In [36]:
bagging_lgbm_model = BaggingClassifier(base_estimator=tuned_lgbm_model, n_estimators=40).fit(all_train_x_dummies, train_y)

[LightGBM] [Warning] feature_fraction is set=0.7973403266487863, colsample_bytree=0.10593113961701743 will be ignored. Current value: feature_fraction=0.7973403266487863
[LightGBM] [Warning] lambda_l1 is set=0.5174759248593073, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5174759248593073
[LightGBM] [Warning] bagging_fraction is set=0.8958797357815458, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8958797357815458
[LightGBM] [Warning] lambda_l2 is set=0.16855495537647786, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.16855495537647786


In [37]:
submission['voted']= list(pd.DataFrame(bagging_lgbm_model.predict_proba(test_x_dummies)).apply(lambda x: x[1], axis=1))

In [38]:
submission.to_csv('./data/bagging_tuned_lgbm.csv')